In [19]:
import os
import json
import sys
import requests
PATENT_BASE_URL = 'https://uspto-documents-storage.s3.amazonaws.com/docs/'
mocks_dir = os.path.join(os.path.abspath(''), 'mocks')
sys.path.insert(0, mocks_dir + '/../../classes/')
#print(mocks_dir + '/../../classes/')
from patent import Patent

In [20]:
data_path = os.path.join(mocks_dir, 'validation_data.json')
data_file = open(data_path)
data = json.load(data_file)

In [21]:
def print_result(patent_number, key, expected, extracted):
    print(key + " is incorrect for patent id " + patent_number)
    print("Extracted: " + extracted)
    print("Expected: " + expected)

def get_patent_xml_path(patent_id):
    ret_path = None
    path_in_mock = os.path.join(mocks_dir, pat_id + '.xml')
    if os.path.exists(path_in_mock):
        ret_path = path_in_mock
    else:
        response = requests.get(PATENT_BASE_URL + patent_id + '.xml')
        if response.status_code == 200:
            with open(path_in_mock, 'wb') as f:
                f.write(response.content)
            ret_path = path_in_mock
    return ret_path

In [22]:
#patent id, expected is subset of extracted, extracted is subset of expected, expected, extracted
import pandas as pd



In [23]:
epitope_results =  pd.DataFrame(columns=['patent_id', 'expected_is_subset', 'extracted_is_subset', 'expected_residues', 'extracted_residues'])

for datum in data:
    pat_id = datum['patentNumber']
    patent_xml_path = get_patent_xml_path(pat_id)
    patent = Patent(patent_xml_path, pat_id)
    
    for key in datum:
        if type(datum[key]) == str:
            if datum[key].lower() != patent.__dict__[key].lower(): 
                print_result(datum['patentNumber'], key, datum[key], patent.__dict__[key])
        elif key == 'mentionedResidues':
            extracted_res = set()
            for a in patent.__dict__[key]:
                if a['location'] == 'claim':
                    for b in a['claimedResidues']:
                        extracted_res.add(b)
            #parsed_epitopes = list(res)
            extracted_res = set(sorted(extracted_res, key=lambda e: int(e)))
            expected_res = set(sorted(datum[key], key=lambda e: int(e)))
            exp_sub = expected_res.issubset(extracted_res)
            ext_sub = extracted_res.issubset(expected_res)
            epitope_results = epitope_results.append(
                {'patent_id': pat_id, 'expected_is_subset': exp_sub, 'extracted_is_subset': ext_sub, 
                'expected_residues': expected_res, 'extracted_residues': extracted_res}, ignore_index=True)
            
        else: 
            if datum[key] != patent.__dict__[key]:
                print_result(datum['patentNumber'], key, str(datum[key]), str(patent.__dict__[key]))


In [24]:
epitope_results

,patent_id,expected_is_subset,extracted_is_subset,expected_residues,extracted_residues
0,US8062640B2,True,True,{},{}
1,US8080243B2,True,True,{},{}
2,US8188234B2,True,True,{},{}
3,US8357371B2,True,True,{},{}
4,US8399646B2,True,True,{},{}
5,US8501184B2,True,True,{},{}
6,US8563698B2,False,True,"{123, 124, 129, 128, 126, 130, 132, 127, 131, ...","{123, 124, 129, 128, 126, 130, 127, 131, 125}"
7,US8829165B2,True,True,"{381, 369, 380, 375, 239, 238, 374, 378, 194, ...","{381, 369, 380, 375, 239, 238, 374, 378, 194, ..."
8,US8889144B2,True,True,{},{}
9,US9175093B2,True,True,{},{}
